# filtering labevents

In [1]:
# DB connection:
import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
%matplotlib inline

###  connect DB

In [2]:
try:
    conn = psycopg2.connect("dbname='mimic' user='mimic' host='localhost' password='meladymimic315'")
except:
    print "I am unable to connect to the database"
print 'connection established'

connection established


### load admissionID , itemID

In [3]:
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

db = np.load('res/itemids.npy').tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

### For each itemid first we check the unit of measurement manually, all of them are ok, no item id has 2 different unit of measurement, both with high number of observations. 
### Then we continue to check if the observations more than 5% is not numeric then we also discard that itemID.
### By discarding, we save it for later process.

In [12]:
valid_lab = []
valid_lab_unit = []
dropped_id = []
for i in lab_itemid:
    cur = conn.cursor()
    cur.execute('SELECT valueuom, count(valueuom) FROM mimiciii.labevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+') group by valueuom')
    outputunits = cur.fetchall()
    outputunits = sorted(outputunits, key=lambda tup: tup[1])
    outputunits.reverse()

    total = 0
    for o in outputunits:
        total += o[1]
    if(total == 0 ): 
        print "LAB "+str(i) + "\t 0 units"
        continue;
    percentage = float(outputunits[0][1]) / total *100.
    
    print "LAB "+str(i) + "\t" + "{:.2f}%".format(percentage) + "\t" + str(outputunits[0][1])+"/"+str(total)+" : "+ str(outputunits)
    
    cur = conn.cursor()
    cur.execute('SELECT count(*) FROM mimiciii.labevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+') and valuenum is null')
    notnum = cur.fetchone()
    
    cur = conn.cursor()
    cur.execute('SELECT count(*) FROM mimiciii.labevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+') and valuenum is not null')
    total = cur.fetchone()
    
    percentage =float(total[0])*100 / (notnum[0]+total[0])
    print "LAB\t" + "{:.4f}".format(percentage)+ "\tNOTNUM=\t" + str(notnum[0]) + "\tNUM=\t" + str(total[0])
    
    if(percentage < 95): 
        dropped_id.append(i);
        print 'dropped\n'
        continue;
        
    print ''
        
    valid_lab.append(i);
    valid_lab_unit.append(outputunits[0][0]);

LAB 51221	100.00%	283305/283305 : [('%', 283305L)]
LAB	99.9944	NOTNUM=	16	NUM=	283289

LAB 50912	100.00%	268395/268395 : [('mg/dL', 268395L)]
LAB	99.9937	NOTNUM=	17	NUM=	268378

LAB 51006	100.00%	267415/267415 : [('mg/dL', 267415L)]
LAB	99.9963	NOTNUM=	10	NUM=	267405

LAB 51301	100.00%	243112/243112 : [('K/uL', 243112L)]
LAB	99.9613	NOTNUM=	94	NUM=	243018

LAB 51222	100.00%	244035/244035 : [('g/dL', 244035L)]
LAB	99.9865	NOTNUM=	33	NUM=	244002

LAB 51249	100.00%	242445/242445 : [('%', 242445L)]
LAB	99.9794	NOTNUM=	50	NUM=	242395

LAB 51279	100.00%	242398/242398 : [('m/uL', 242398L)]
LAB	99.9398	NOTNUM=	146	NUM=	242252

LAB 51250	100.00%	242392/242392 : [('fL', 242392L)]
LAB	99.9389	NOTNUM=	148	NUM=	242244

LAB 51248	100.00%	242395/242395 : [('pg', 242395L)]
LAB	99.9398	NOTNUM=	146	NUM=	242249

LAB 51277	100.00%	242367/242367 : [('%', 242367L)]
LAB	99.9422	NOTNUM=	140	NUM=	242227

LAB 51265	100.00%	249171/249171 : [('K/uL', 249171L)]
LAB	99.9864	NOTNUM=	34	NUM=	249137

LAB 50971	100.00%

### Save the id that unit of measurement are consistent and are numeric.

In [5]:
np.save('res/filtered_lab',{'id':valid_lab,'unit':valid_lab_unit});
print "saved !";

saved !


### From the ItemID that we discard in first part, for each itemID, we list all of its distinct values and number of the observations for that values, list just top 20.

### The values in this table is very dirty, In this step, I use the result here to manually mark them as a categorical or numerical feature, and process them in next step.

In [25]:
for d in dropped_id:
    print 'LAB : ' + str(d) 
    cur = conn.cursor()
    cur.execute('SELECT value, valueuom, count(*) as x FROM mimiciii.labevents as lb WHERE itemid = '+ str(d) +' and hadm_id in ('+admission_ids_txt+') GROUP BY value, valueuom ORDER BY x DESC')
    droped_outs = cur.fetchall()
    drop_array = [];
    ct =0
    total = 0;
    for dx in droped_outs:
        total += dx[2]
    print "Count ",total
    for dx in droped_outs:
        ct+=1
        if(ct>20):
            break;
        dx = list(dx)
        print dx[1],dx[0],"\t",dx[2]
    print ''



LAB : 51519
Count  27107
 NONE 	24732
 FEW 	651
 MANY 	603
 MOD 	508
 RARE 	316
 OCC 	278
   	13
 N 	4
 F 	2

LAB : 51492
Count  26518
mg/dL NEG 	7600
mg/dL 30 	7560
mg/dL TR 	3309
mg/dL 100 	2945
mg/dL 25 	2002
mg/dL 75 	1309
mg/dL 500 	493
mg/dL 300 	433
mg/dL 150 	398
mg/dL >300 	351
mg/dL 600 	63
mg/dL   	35
mg/dL >600 	14
mg/dL 15 	5
mg/dL NEGATIVE 	1

LAB : 51516
Count  25640
#/hpf 0-2 	4947
#/hpf 1 	2581
#/hpf 3-5 	2332
#/hpf 2 	1746
#/hpf 0 	1515
#/hpf 3 	1273
#/hpf 6-10 	1078
#/hpf 4 	846
#/hpf >50 	809
#/hpf 5 	694
#/hpf 11-20 	683
#/hpf 6 	503
#/hpf 21-50 	497
#/hpf 7 	409
#/hpf 8 	367
#/hpf 9 	311
#/hpf 10 	280
#/hpf <1 	252
#/hpf 11 	248
#/hpf 12 	219

LAB : 51493
Count  25253
#/hpf 0-2 	4250
#/hpf 1 	2842
#/hpf 0 	1953
#/hpf 3-5 	1791
#/hpf 2 	1617
#/hpf >50 	1241
#/hpf 6-10 	1171
#/hpf 3 	915
#/hpf 11-20 	850
#/hpf 21-50 	727
#/hpf 4 	617
#/hpf 5 	532
#/hpf 6 	386
#/hpf 7 	355
#/hpf 8 	294
#/hpf <1 	269
#/hpf 9 	240
#/hpf 10 	211
#/hpf 11 	209
#/hpf 12 	167

LAB : 51476


# This part is to save the main unit for numeric itemID that is manually marked in previous section. 

### also I print something more to see some statistic and check correctness.

In [26]:
valid_lab_num = [51475,50845,51280,50935,51479,50922,51501,50925,50856,50981,51213,50915,51497,51046,50835,51176,51180,51194,51196,51130,51131,51132,50906,51076,51422,51517,51229,50946,51471,50899,51515,51369,50992,50958,50926,50961,51228,50877,51494,50990,50991,51489,51488,51061,51225,50894,50989,51209,51516,51493,51476,50911,51003,51482]
valid_lab_cate = [51519,51461,51495,51390,51096,51403,51391,51189,51405,50901,51407,51089,51153,51171,51220,51179,51468,51016,51401,51207,51472,51394,51291,51500,51161,51286,51410,51417,51389,51195,51322,51201,51142,51311,51135,51329,51396,51318,51421,50875,51485,51316,51308,51056,51537,51414,51304,51157,51079,51075,51071,51074,51092,51090,51512,50828,50933,51266,51246,51267,51137,51252,51268,51233,50955,50887,51523,51462,50979,51260,50919,51287,51296,51151,51474,51107,51103,51236,50940,50943,50941,51145,51294,50942,51240,51292,51518,50873,51505,51469,50975,51424,51134,51411,50944,50937,51197,51425,51426,51098,51243,51373,51147,51085,51216,51400,51388,51412,50872,51150,51423,51402,50938,50939,51234,51420,51338,51325,51183,51164,50948,51313,50857,51399,51239,51238,51416,51319,51230,51337,51152,51168,51184,51341,51340,51198,51326,51303,51315,50876,51261,51499,50871,51086,51192,51167,51332,51314,51342,51231,51321,51264,51374,51370,51372,51307,51235,51215,51317,51503,51335,51172,50874,51219,51305,51310,51323,51334,51320,51178,51156,51309,51306,51155,51324,51177,51158,51328,50913,51336,51159,51333,51154,51331,51262,51339,51182,51217,50918,51193,51460,51191,51510,51091,51465,51017,51160,51408,51418,51190,51187,51413,50932,51511,51350,51393,51481,51392,51490,51397,51470,51247,51312,51395,51295,51173,51502,51415,51508,51506,51466,51486,51464,51487,51463,50920,50800,50880,50879,50999,50812]
valid_lab_num_unit = []
for i in valid_lab_num:
        
    cur = conn.cursor()
    cur.execute('SELECT valueuom, count(valueuom) FROM mimiciii.labevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+') group by valueuom')
    outputunits = cur.fetchall()
    outputunits = sorted(outputunits, key=lambda tup: tup[1])
    outputunits.reverse()
    total = 0
    for o in outputunits:
        total += o[1]
    if(total == 0 ): continue;
    percentage = float(outputunits[0][1]) / total *100.
    print "LAB "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)
    
    cur = conn.cursor()
    cur.execute('SELECT count(*) FROM mimiciii.labevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+') and valuenum is null')
    notnum = cur.fetchone()
    
    cur = conn.cursor()
    cur.execute('SELECT count(*) FROM mimiciii.labevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+')')
    total = cur.fetchone()
    
    percentage =float(total[0] -notnum[0])*100 / total[0]
    print "LAB "+str(i) + " : "+ "{:.4f}".format(percentage)+ " : NOTNUM = " + str(notnum[0]) + " : ALL " + str(total[0])
    
    if(percentage < 95): 
        dropped_id.append(i);
        cur = conn.cursor()
        cur.execute('SELECT distinct(value,valuenum,valueuom) FROM mimiciii.labevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+') limit 20')
        droped_outs = cur.fetchall()
        drop_array = [];
        for d in droped_outs:
            drop_array.append(d)
        print drop_array

    print ''
    sys.stdout.flush()   
    valid_lab_num_unit.append(outputunits[0][0]);

LAB 51475 : 100.00 : 1 : [('#/lpf', 1L)]
LAB 51475 : 0.0000 : NOTNUM = 1 : ALL 1
[('(<1,,#/lpf)',)]

LAB 50845 : 100.00 : 1 : [('', 1L)]
LAB 50845 : 0.0000 : NOTNUM = 1 : ALL 1
[('("ALKP = 75",,"")',)]

LAB 51280 : 100.00 : 1 : [('sec', 2L)]
LAB 51280 : 50.0000 : NOTNUM = 1 : ALL 2
[('(>150,,sec)',), ('(61.4,61.4,sec)',)]

LAB 50935 : 100.00 : 1 : [('mg/dL', 3853L)]
LAB 50935 : 92.2139 : NOTNUM = 300 : ALL 3853
[('(10,10,mg/dL)',), ('(100,100,mg/dL)',), ('(101,101,mg/dL)',), ('(102,102,mg/dL)',), ('(103,103,mg/dL)',), ('(104,104,mg/dL)',), ('(105,105,mg/dL)',), ('(106,106,mg/dL)',), ('(107,107,mg/dL)',), ('(108,108,mg/dL)',), ('(109,109,mg/dL)',), ('(11,11,mg/dL)',), ('(110,110,mg/dL)',), ('(111,111,mg/dL)',), ('(112,112,mg/dL)',), ('(113,113,mg/dL)',), ('(114,114,mg/dL)',), ('(115,115,mg/dL)',), ('(116,116,mg/dL)',), ('(117,117,mg/dL)',)]

LAB 51479 : 100.00 : 1 : [('#/lpf', 2877L)]
LAB 51479 : 59.5759 : NOTNUM = 1163 : ALL 2877
[('(0-2,,#/lpf)',), ('("0-2 COARSE",,#/lpf)',), ('("0-2,

## Save it ! 

In [28]:
np.save('res/filtered_lab_num',{'id':valid_lab_num,'unit':valid_lab_num_unit});
np.save('res/filtered_lab_cate',{'id':valid_lab_cate,'unit':None});
print "saved !";

saved !


In [16]:
valid_lab_num

NameError: name 'valid_lab_num' is not defined

In [10]:
len(lab_itemid)

729

In [9]:
len(valid_lab)

372

In [11]:
len(dropped_id)

298

In [13]:
59

59

In [15]:
372+298

670

In [17]:
valid_lab_num = [51475,50845,51280,50935,51479,50922,51501,50925,50856,50981,51213,50915,51497,51046,50835,51176,51180,51194,51196,51130,51131,51132,50906,51076,51422,51517,51229,50946,51471,50899,51515,51369,50992,50958,50926,50961,51228,50877,51494,50990,50991,51489,51488,51061,51225,50894,50989,51209,51516,51493,51476,50911,51003,51482]
valid_lab_cate = [51519,51461,51495,51390,51096,51403,51391,51189,51405,50901,51407,51089,51153,51171,51220,51179,51468,51016,51401,51207,51472,51394,51291,51500,51161,51286,51410,51417,51389,51195,51322,51201,51142,51311,51135,51329,51396,51318,51421,50875,51485,51316,51308,51056,51537,51414,51304,51157,51079,51075,51071,51074,51092,51090,51512,50828,50933,51266,51246,51267,51137,51252,51268,51233,50955,50887,51523,51462,50979,51260,50919,51287,51296,51151,51474,51107,51103,51236,50940,50943,50941,51145,51294,50942,51240,51292,51518,50873,51505,51469,50975,51424,51134,51411,50944,50937,51197,51425,51426,51098,51243,51373,51147,51085,51216,51400,51388,51412,50872,51150,51423,51402,50938,50939,51234,51420,51338,51325,51183,51164,50948,51313,50857,51399,51239,51238,51416,51319,51230,51337,51152,51168,51184,51341,51340,51198,51326,51303,51315,50876,51261,51499,50871,51086,51192,51167,51332,51314,51342,51231,51321,51264,51374,51370,51372,51307,51235,51215,51317,51503,51335,51172,50874,51219,51305,51310,51323,51334,51320,51178,51156,51309,51306,51155,51324,51177,51158,51328,50913,51336,51159,51333,51154,51331,51262,51339,51182,51217,50918,51193,51460,51191,51510,51091,51465,51017,51160,51408,51418,51190,51187,51413,50932,51511,51350,51393,51481,51392,51490,51397,51470,51247,51312,51395,51295,51173,51502,51415,51508,51506,51466,51486,51464,51487,51463,50920,50800,50880,50879,50999,50812]


In [19]:
print len(valid_lab_num)
print len(valid_lab_cate)


54
231


In [20]:
298-54-231

13